In [1]:
""" A model is designed which can classify hand gestures as Stone, Paper & Scissors using "rock_paper_scissors" dataset. This model is then
deployed on an Android device on an app. """

try:
  %tensorflow_version 2.x
except:
  pass

# Imports 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

TensorFlow 2.x selected.


In [2]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [3]:
# We use subsplit here because dataset by itself does not define the standard splits and it has to be split manually into train,
# validation, test set
splits = tfds.Split.ALL.subsplit(weighted=(80,10,10))
(train_dataset, validation_dataset, test_dataset), info = tfds.load("rock_paper_scissors", as_supervised=True, with_info=True, split=splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/1.0.0. Subsequent calls will reuse this data.


In [0]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE)/255.0
  return image, label

In [0]:
BATCH_SIZE = 32
train_batches = train_dataset.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_dataset.map(format_image).batch(BATCH_SIZE)
test_batches = test_dataset.batch(1)

In [6]:
# Inspecting a batch
for img,label in train_batches.take(5):
  pass
img.shape

TensorShape([32, 224, 224, 3])

In [7]:
# I've kept fine-tuning off for this exercise
model = tf.keras.Sequential([
        hub.KerasLayer(MODULE_HANDLE,
                       input_shape = IMAGE_SIZE + (3,),
                       output_shape = [FV_SIZE],
                       trainable = False),
        tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
# If fine-tuning is ON, use optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9)
model.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [10]:
EPOCHS = 3
hist = model.fit(
    train_batches,
    validation_data = validation_batches,
    epochs = EPOCHS
)

Epoch 1/3
73/73 [==============================] - 17s 226ms/step - loss: 0.3716 - accuracy: 0.8907 - val_loss: 0.1257 - val_accuracy: 0.9857
Epoch 2/3
73/73 [==============================] - 8s 111ms/step - loss: 0.0798 - accuracy: 0.9966 - val_loss: 0.0662 - val_accuracy: 0.9964
Epoch 3/3
73/73 [==============================] - 9s 117ms/step - loss: 0.0448 - accuracy: 0.9991 - val_loss: 0.0421 - val_accuracy: 1.0000


In [18]:
RPS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model,RPS_SAVED_MODEL)

INFO:tensorflow:Assets written to: exp_saved_model/assets


INFO:tensorflow:Assets written to: exp_saved_model/assets


In [20]:
# To verify default signature of saved model----Input/Output dimensions,dtype of saved model
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [22]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 3), dtype=tf.float32, name='dense')}
